In [14]:
from django_for_jupyter import init_django
init_django("mysite")

In [24]:
from Profiles.models import Post, Like, Profile,Comment
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [25]:
profiles = Profile.objects.all()
posts = Post.objects.all()

In [26]:
profile_index = {profile.id: idx for idx, profile in enumerate(profiles)}
post_index = {post.id: idx for idx, post in enumerate(posts)}

In [46]:
profile_index

{2: 0,
 5: 1,
 6: 2,
 7: 3,
 8: 4,
 9: 5,
 10: 6,
 11: 7,
 12: 8,
 13: 9,
 14: 10,
 15: 11,
 16: 12,
 17: 13,
 18: 14,
 19: 15,
 20: 16,
 21: 17,
 22: 18,
 23: 19,
 24: 20,
 25: 21,
 26: 22,
 27: 23,
 28: 24,
 29: 25,
 30: 26,
 31: 27,
 32: 28,
 33: 29,
 34: 30,
 35: 31,
 36: 32,
 37: 33,
 38: 34,
 39: 35,
 40: 36,
 41: 37,
 42: 38,
 43: 39,
 44: 40,
 45: 41,
 46: 42,
 47: 43,
 48: 44,
 49: 45,
 50: 46,
 51: 47,
 52: 48,
 53: 49,
 54: 50,
 55: 51,
 56: 52,
 57: 53,
 58: 54,
 59: 55,
 60: 56,
 61: 57,
 62: 58,
 63: 59,
 64: 60,
 65: 61,
 66: 62,
 67: 63,
 68: 64,
 69: 65,
 70: 66,
 71: 67,
 72: 68,
 73: 69,
 74: 70,
 75: 71,
 76: 72,
 77: 73,
 78: 74,
 79: 75,
 80: 76,
 81: 77,
 82: 78,
 83: 79,
 84: 80,
 85: 81,
 86: 82,
 87: 83,
 88: 84,
 89: 85,
 90: 86,
 91: 87,
 92: 88,
 93: 89,
 94: 90,
 95: 91,
 96: 92,
 97: 93,
 98: 94,
 99: 95,
 100: 96,
 101: 97,
 102: 98,
 103: 99,
 104: 100,
 105: 101,
 106: 102,
 107: 103,
 108: 104,
 109: 105,
 110: 106,
 111: 107,
 112: 108,
 113: 109,
 1

In [23]:
user_post_matrix = np.zeros((len(profiles), len(posts)))

In [47]:
likes = Like.objects.filter(enabled=True).order_by('-updated_at')[:200]
for like in likes:
    user_idx = profile_index[like.profile.id]
    post_idx = post_index[like.post.id]
    user_post_matrix[user_idx,post_idx] = 1

In [48]:
user_post_df = pd.DataFrame(user_post_matrix,index=[profile.id for profile in profiles], columns = [post.id for post in posts])

In [49]:
user_post_df.tail()

,3,4,5,6,8,9,11,21,22,23,...,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808
475,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
476,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
477,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
478,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
479,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
svd = TruncatedSVD(n_components=10)
user_factors = svd.fit_transform(user_post_matrix)
post_factors = svd.components_.T

user_factors.shape


(476, 10)

In [51]:
post_factors.shape

(1795, 10)

In [52]:
def recommend_posts(user_id, top_n=5):
    user_idx = profile_index[user_id]
    
    user_factor = user_factors[user_idx].reshape(1, -1)  

    scores = np.dot(user_factor, post_factors.T)
    
    top_post_indices = np.argsort(scores.flatten())[-top_n:][::-1] 
    
    recommended_post_ids = [list(post_index.keys())[idx] for idx in top_post_indices]
    
    return recommended_post_ids



In [53]:
user_id = profiles[0].id  
recommended_posts = recommend_posts(user_id, top_n=100)
print(f"Recommended post IDs for {Profile.objects.get(id=user_id).user}:", recommended_posts)


Recommended post IDs for admin: [3, 5, 239, 318, 72, 48, 218, 277, 315, 21, 120, 252, 197, 107, 57, 295, 84, 322, 169, 75, 50, 62, 8, 305, 171, 44, 288, 69, 47, 291, 327, 54, 111, 9, 137, 333, 76, 191, 164, 198, 330, 74, 31, 141, 182, 234, 115, 33, 55, 208, 138, 210, 241, 73, 150, 256, 217, 167, 172, 61, 155, 203, 81, 39, 228, 49, 127, 78, 227, 109, 22, 219, 65, 91, 253, 281, 86, 195, 188, 149, 97, 199, 312, 317, 67, 303, 94, 11, 168, 232, 71, 179, 314, 123, 114, 268, 272, 211, 301, 27]
